In [1]:
from flask import Flask, jsonify, render_template
import requests  # Importe a biblioteca 'requests' para fazer chamadas HTTP externas
from dotenv import load_dotenv
import os
import pandas as pd
import sqlite3

In [2]:
cidade = "Kyoto"
load_dotenv()
API_KEY = os.getenv("API_KEY")
URL = f"http://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={API_KEY}&units=metric&lang=pt_br"
response = requests.get(URL)
dados_json = response.json()


In [3]:
df_DadosClima = pd.DataFrame([dados_json])
print(df_DadosClima)


                               coord  \
0  {'lon': 135.7538, 'lat': 35.0211}   

                                             weather      base  \
0  [{'id': 804, 'main': 'Clouds', 'description': ...  stations   

                                                main  visibility  \
0  {'temp': 10.04, 'feels_like': 9.24, 'temp_min'...       10000   

                                        wind        clouds          dt  \
0  {'speed': 0.89, 'deg': 334, 'gust': 1.34}  {'all': 100}  1743618319   

                                                 sys  timezone       id  \
0  {'type': 2, 'id': 265569, 'country': 'JP', 'su...     32400  1857910   

    name  cod  
0  Kyoto  200  


In [4]:
class Api_requi:
    def __init__(self,api_key,data_return,cidade_api,url_api):
        self.api_key = api_key
        self.data_return = data_return
        self.cidade_api = cidade_api
        self.url_api = url_api

cidade_get = "Moscow"
api_getcord = Api_requi(API_KEY,[],cidade_get,"")
URL = f"http://api.openweathermap.org/data/2.5/weather?q={cidade_get}&appid={API_KEY}&units=metric&lang=pt_br"
response_route = requests.get(URL)
api_getcord.data_return = response_route.json()
pd_data_filtrado = []
pd_outData = pd.DataFrame([api_getcord.data_return])

lat_for_api = pd_outData["coord"][0]['lat']
lon_for_api = pd_outData["coord"][0]['lon']
api_previsao = Api_requi(API_KEY,[],cidade_get,"")
api_previsao.url_api = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat_for_api}&lon={lon_for_api}&units=metric&cnt=40&appid={API_KEY}"
response_prev = requests.get(api_previsao.url_api)
response_prev_json = response_prev.json()
print(response_prev_json)

df_list = pd.json_normalize(response_prev_json['list'], sep='_')  # Separação com '_'

# Processar a coluna 'weather' (lista de dicionários)
weather_normalized = pd.json_normalize(df_list['weather'].apply(lambda x: x[0] if x else {}))
weather_normalized = weather_normalized.add_prefix('weather_')
df_list = pd.concat([df_list.drop('weather', axis=1), weather_normalized], axis=1)

# Normalizar os dados da cidade (city) e adicionar prefixo
df_city = pd.json_normalize(response_prev_json['city'], sep='_').add_prefix('city_')

# Combinar os dados (como há apenas uma cidade, cruzamos os dados)
df_final = pd.merge(df_list, df_city, how='cross') 

# Renomear colunas (substituir '.' por '_', redundante aqui devido ao sep='_')
df_final.columns = df_final.columns.str.replace('.', '_')

# Resultado final
df_final






#print(df2)
print(df_final)




{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1743627600, 'main': {'temp': 8.73, 'feels_like': 7.09, 'temp_min': 7.88, 'temp_max': 8.73, 'pressure': 1017, 'sea_level': 1017, 'grnd_level': 998, 'humidity': 55, 'temp_kf': 0.85}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'clouds': {'all': 1}, 'wind': {'speed': 2.83, 'deg': 311, 'gust': 8.42}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-04-02 21:00:00'}, {'dt': 1743638400, 'main': {'temp': 8.18, 'feels_like': 6.02, 'temp_min': 7.09, 'temp_max': 8.18, 'pressure': 1017, 'sea_level': 1017, 'grnd_level': 998, 'humidity': 60, 'temp_kf': 1.09}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'clouds': {'all': 12}, 'wind': {'speed': 3.51, 'deg': 315, 'gust': 9.18}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-04-03 00:00:00'}, {'dt': 1743649200, 'main': {'temp': 6.7, 'feels_like': 4.15, 'temp_min': 5.69, 'te

In [8]:
db = sqlite3.connect("data_weather.db")
cursor = db.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS weather_data_forecast(
id INTEGER PRIMARY KEY AUTOINCREMENT,
cidade VARCHAR(255) NOT NULL UNIQUE,
temperatura VARCHAR(5) NOT NULL,
umidade FLOAT NOT NULL,
description VARCHAR(255) NOT NULL
)

""")


db = sqlite3.connect("data_weather.db")
cursor = db.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS weather_data_daily(

cidade VARCHAR(255) NOT NULL UNIQUE,
temperatura VARCHAR(5) NOT NULL,
umidade FLOAT NOT NULL,
description VARCHAR(255) NOT NULL
)
""")

NameError: name 'pd_response_prev' is not defined